In [4]:
import requests
from bs4 import BeautifulSoup
import lxml
import pandas as pd
import time

In [5]:
session = requests.Session()

In [6]:
session.headers

{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate, br', 'Accept': '*/*', 'Connection': 'keep-alive'}

In [7]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; en-US; rv:76.0) Gecko/20100101 Firefox/76.0'}
session.headers.update(headers)

In [8]:
params_search = {
    'fillQuickSearch':'false',
    'target':'advanced',
    'expand':'dl',
    'field1':'AllField',
    'text1':"Text AND Mining OR Clustering OR Classification OR Machine AND Learning OR Data AND Mining",
    'Ppub':"[20170315 TO 20220315]",
    'pageSize':50,
    'startPage':0,
    'rel':'nofollow',
    'ContentItemType':'research-article'
}

In [10]:
response = session.get('https://dl.acm.org/action/doSearch',params=params_search)

In [11]:
response.status_code

200

In [12]:
response.url

'https://dl.acm.org/action/doSearch?fillQuickSearch=false&target=advanced&expand=dl&field1=AllField&text1=Text+AND+Mining+OR+Clustering+OR+Classification+OR+Machine+AND+Learning+OR+Data+AND+Mining&Ppub=%5B20170315+TO+20220315%5D&pageSize=50&startPage=0&rel=nofollow&ContentItemType=research-article'

In [34]:
pages = 40
soup = BeautifulSoup(response.text, 'lxml')
count_articles = int(''.join(soup.select_one('.hitsLength').text.split(','))) // 50
if count_articles < pages:
    pages = count_articles

In [27]:
def parse_page(params, page):
    params['startPage'] = page
    titles_links = [[title.text, f"https://dl.acm.org{title.select_one('a')['href']}"] for title in soup.select('.hlFld-Title')]
    return titles_links

In [41]:
def parse_info(url):
    response = session.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    authors = [author.text for author in soup.select('.loa__author-name')]
    meta = soup.select_one('.issue-item__detail a')['title']
    date = soup.select_one('.epub-section__date')
    if date is None:
        date = ''
    else:
        date = date.text.strip()
    abstract = soup.select_one('.abstractSection').text.strip()
    return authors, meta, date, abstract

In [ ]:
all_articles = {'title': [], 'authors': [], 'date': [], 'source': [], 'abstract': [], 'link': []}
for page in range(pages):
    titles_links = parse_page(params_search, page)
    for title, link in titles_links:
        authors, meta, date, abstract = parse_info(link)
        all_articles['title'] += [title]
        all_articles['authors'] += [authors]
        all_articles['date'] += [date]
        all_articles['source'] += [meta]
        all_articles['abstract'] += [abstract]
        all_articles['link'] += [link]